In [3]:
import pandas as pd
import numpy as np

In [4]:
data = pd.read_pickle("../../data/processed/data_cleaned.pkl.gz", compression='gzip')

In [5]:
data.sample(20)

,searchDate,flightDate,startingAirport,destinationAirport,isBasicEconomy,isRefundable,isNonStop,totalFare,totalTravelDistance,segmentsDepartureTimeEpochSeconds_Leg1,...,segmentsDurationInSeconds_Leg4,segmentsDistance_Leg1,segmentsDistance_Leg2,segmentsDistance_Leg3,segmentsDistance_Leg4,segmentsCabinCode_Leg1,segmentsCabinCode_Leg2,segmentsCabinCode_Leg3,segmentsCabinCode_Leg4,traveltime_hours
9738944,2022-04-25,2022-05-10,MIA,DTW,False,False,False,187.199997,1589.0,1.652179e+09,...,0.0,1104.0,485.0,0.0,0.0,coach,coach,0,0,11.216667
5806467,2022-05-18,2022-06-06,EWR,DEN,False,False,False,398.589996,1627.0,1.654549e+09,...,0.0,485.0,1142.0,0.0,0.0,coach,coach,0,0,8.100000
9734482,2022-04-25,2022-05-03,MIA,DEN,False,False,False,147.600006,1834.0,1.651574e+09,...,0.0,958.0,876.0,0.0,0.0,coach,coach,0,0,7.083333
4651284,2022-04-27,2022-05-17,DTW,PHL,False,False,False,217.610001,912.0,1.652810e+09,...,0.0,240.0,672.0,0.0,0.0,coach,coach,0,0,7.683333
1348808,2022-04-30,2022-06-17,BOS,ORD,False,False,True,205.600006,862.0,1.655465e+09,...,0.0,862.0,0.0,0.0,0.0,coach,0,0,0,2.833333
1667274,2022-05-09,2022-05-13,BOS,OAK,False,False,False,728.599976,2944.0,1.652443e+09,...,0.0,2606.0,338.0,0.0,0.0,coach,coach,0,0,10.216667
2258352,2022-04-30,2022-05-22,CLT,DFW,False,False,False,614.099976,1391.0,1.653255e+09,...,0.0,592.0,799.0,0.0,0.0,coach,coach,0,0,6.483333
7568777,2022-04-27,2022-04-29,LAX,SFO,False,False,True,170.600006,351.0,1.651273e+09,...,0.0,351.0,0.0,0.0,0.0,coach,0,0,0,1.550000
12969572,2022-04-29,2022-06-01,SFO,IAD,False,False,False,567.210022,2991.0,1.654098e+09,...,0.0,679.0,2312.0,0.0,0.0,coach,coach,0,0,12.183333
3574260,2022-04-19,2022-05-08,DFW,PHL,False,False,False,301.600006,1304.0,1.652011e+09,...,0.0,1177.0,127.0,0.0,0.0,coach,coach,0,0,4.650000


In [6]:
data.shape

(13519999, 58)

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13519999 entries, 0 to 13519998
Data columns (total 58 columns):
 #   Column                                  Dtype   
---  ------                                  -----   
 0   searchDate                              object  
 1   flightDate                              object  
 2   startingAirport                         category
 3   destinationAirport                      category
 4   isBasicEconomy                          bool    
 5   isRefundable                            bool    
 6   isNonStop                               bool    
 7   totalFare                               float32 
 8   totalTravelDistance                     float32 
 9   segmentsDepartureTimeEpochSeconds_Leg1  float32 
 10  segmentsDepartureTimeEpochSeconds_Leg2  float32 
 11  segmentsDepartureTimeEpochSeconds_Leg3  float32 
 12  segmentsDepartureTimeEpochSeconds_Leg4  float32 
 13  segmentsDepartureTimeRaw_Leg1           object  
 14  segmentsDepartur

In [8]:
data['flightDate'] = pd.to_datetime(data['flightDate'])

In [9]:
data['flightMonth'] = data['flightDate'].dt.month
data['flightDay'] = data['flightDate'].dt.day
data['flightDayOfWeek'] = data['flightDate'].dt.dayofweek
data['flightWeekOfYear'] = data['flightDate'].dt.isocalendar().week
data['flightIsWeekend'] = data['flightDate'].dt.dayofweek >= 5

In [10]:
data['cabin_type'] = data['segmentsCabinCode_Leg1']

In [11]:
data = data.dropna(subset=['traveltime_hours'])

In [12]:
data = data[data['totalTravelDistance'] != 0]

In [13]:
data['segmentsDepartureTimeRaw_Leg1'] = pd.to_datetime(data['segmentsDepartureTimeRaw_Leg1'], utc=True)

In [14]:
# Defining a function to categorize the time of day
def categorize_time_of_day(hour):
    if 5 <= hour < 12:
        return 'morning'
    elif 12 <= hour < 17:
        return 'afternoon'
    elif 17 <= hour < 21:
        return 'evening'
    else:
        return 'night'

In [15]:
data['departure_time'] = data['segmentsDepartureTimeRaw_Leg1'].dt.hour.apply(categorize_time_of_day)

In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12285686 entries, 0 to 13519918
Data columns (total 65 columns):
 #   Column                                  Dtype              
---  ------                                  -----              
 0   searchDate                              object             
 1   flightDate                              datetime64[ns]     
 2   startingAirport                         category           
 3   destinationAirport                      category           
 4   isBasicEconomy                          bool               
 5   isRefundable                            bool               
 6   isNonStop                               bool               
 7   totalFare                               float32            
 8   totalTravelDistance                     float32            
 9   segmentsDepartureTimeEpochSeconds_Leg1  float32            
 10  segmentsDepartureTimeEpochSeconds_Leg2  float32            
 11  segmentsDepartureTimeEpochSeconds_L

In [17]:
selected_columns = [
    'startingAirport',
    'destinationAirport',
    'totalFare',
    'totalTravelDistance',
    'traveltime_hours',
    'flightMonth',
    'flightDay',
    'flightDayOfWeek',
    'flightWeekOfYear',
    'flightIsWeekend',
    'cabin_type',
    'departure_time'
]

final_data = data[selected_columns]

In [ ]:
final_data.sample(10)

In [ ]:
final_data.info()

In [ ]:
#Modelling

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import GridSearchCV
import joblib

In [19]:
X = final_data.drop('totalFare', axis=1)
y = final_data['totalFare']

In [20]:
label_cols = ['startingAirport', 'destinationAirport', 'cabin_type', 'departure_time']

In [21]:
label_encoders = {}
for col in label_cols:
    label_encoders[col] = LabelEncoder()
    X[col] = label_encoders[col].fit_transform(X[col])

In [22]:
X_train, X_, y_train, y_ = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_, y_, test_size=0.5, random_state=42)

In [ ]:
#Baseline model

In [23]:
mean_total_fare = y_train.mean()

baseline_predictions = [mean_total_fare] * len(X_test)

rmse_baseline = np.sqrt(mean_squared_error(y_test, baseline_predictions))
print(f"Root Mean Squared Error of the baseline model: {rmse_baseline}")

Root Mean Squared Error of the baseline model: 209.85630798339844


In [24]:
categorical_cols = X.select_dtypes(include=['category', 'object', 'bool']).columns
encoder = OneHotEncoder(handle_unknown='ignore')
X_train_encoded = encoder.fit_transform(X_train[categorical_cols])
X_val_encoded = encoder.transform(X_val[categorical_cols])
X_test_encoded = encoder.transform(X_test[categorical_cols])

In [25]:
X_train_processed = np.concatenate([X_train_encoded.toarray(), X_train.drop(categorical_cols, axis=1).values], axis=1)
X_val_processed = np.concatenate([X_val_encoded.toarray(), X_val.drop(categorical_cols, axis=1).values], axis=1)
X_test_processed = np.concatenate([X_test_encoded.toarray(), X_test.drop(categorical_cols, axis=1).values], axis=1)

In [26]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_processed)
X_val_scaled = scaler.transform(X_val_processed)
X_test_scaled = scaler.transform(X_test_processed)

In [27]:
X_train.shape, X_test.shape, X_val.shape

((8599980, 11), (1842853, 11), (1842853, 11))

In [28]:
import sys
sys.path.append('../../src/models')

In [ ]:
#exp 1 XGBOOST with default parameters

In [29]:
xgb_default_model = XGBRegressor()
xgb_default_model.fit(X_train_scaled, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [30]:
train_predictions_xgb_default = xgb_default_model.predict(X_train_scaled)
test_predictions_xgb_default = xgb_default_model.predict(X_test_scaled)
val_predictions_xgb_default = xgb_default_model.predict(X_val_scaled)

In [31]:
train_rmse_xgb_default = mean_squared_error(y_train, train_predictions_xgb_default, squared=False)
test_rmse_xgb_default = mean_squared_error(y_test, test_predictions_xgb_default, squared=False)
val_rmse_xgb_default = mean_squared_error(y_test, val_predictions_xgb_default, squared=False)

In [32]:
print(f"Training RMSE: {train_rmse_xgb_default}")
print(f"Test RMSE: {test_rmse_xgb_default}")
print(f"Val RMSE: {val_rmse_xgb_default}")

Training RMSE: 115.89431762695312
Test RMSE: 115.97388458251953
Val RMSE: 268.9998779296875


In [33]:
joblib.dump(xgb_default_model, '../../models/ANIKA/xgb_default_model.joblib')

['../../models/ANIKA/xgb_default_model.joblib']

In [ ]:
#exp 2 XGBOOST with best parameters

In [34]:
xgb_tuned_model = XGBRegressor()
param_grid_xgb_tuned = {
    'n_estimators': [150],  
    'max_depth': [3, 4, 5],          
    'learning_rate': [0.01, 0.1, 0.2]  
}

In [35]:
grid_search__xgb_tuned = GridSearchCV(xgb_tuned_model, param_grid_xgb_tuned, cv=5, scoring='neg_mean_squared_error')
grid_search__xgb_tuned.fit(X_train_scaled, y_train)

GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None, gpu_id=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, m...
                                    max_cat_threshold=None,
                                    max_cat_to_onehot=None, max_delta_step=None,
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None, n_estimators=100,
                                    n_jobs=None, num_parallel_tree=None,
                                    predictor=None, random_state=None, ...),
             param_grid={'learning_rate': [0.01, 0.1, 0.2],
                         'max_depth': [3, 4, 5], 'n_estimators': [150]},
             scoring='neg_mean_squared_error')

In [36]:
best_params__xgb_tuned = grid_search__xgb_tuned.best_params_
print("Best Hyperparameters:", best_params__xgb_tuned)

Best Hyperparameters: {'learning_rate': 0.2, 'max_depth': 5, 'n_estimators': 150}


In [37]:
xgb_tuned_model = XGBRegressor(**best_params__xgb_tuned)
xgb_tuned_model.fit(X_train_scaled, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.2, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=150, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [38]:
train_predictions_xgb_tuned = xgb_tuned_model.predict(X_train_scaled)
test_predictions_xgb_tuned = xgb_tuned_model.predict(X_test_scaled)
val_predictions_xgb_tuned = xgb_tuned_model.predict(X_val_scaled)

In [39]:
train_rmse_xgb_tuned = mean_squared_error(y_train, train_predictions_xgb_tuned, squared=False)
test_rmse_xgb_tuned = mean_squared_error(y_test, test_predictions_xgb_tuned, squared=False)
val_rmse_xgb_tuned = mean_squared_error(y_val, val_predictions_xgb_tuned, squared=False)

In [40]:
print(f"Training RMSE: {train_rmse_xgb_tuned}")
print(f"Test RMSE: {test_rmse_xgb_tuned}")
print(f"Val RMSE: {val_rmse_xgb_tuned}")

Training RMSE: 120.94856262207031
Test RMSE: 120.98704528808594
Val RMSE: 121.26666259765625


In [41]:
joblib.dump(xgb_tuned_model, '../../models/ANIKA/xgb_tuned_model.joblib')

['../../models/ANIKA/xgb_tuned_model.joblib']

In [ ]:
#exp 3 LightGBM with default parameters

In [42]:
from lightgbm import LGBMRegressor
lgb_default_model = LGBMRegressor()
lgb_default_model.fit(X_train_scaled, y_train)

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.132513 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 620
[LightGBM] [Info] Number of data points in the train set: 8599980, number of used features: 12
[LightGBM] [Info] Start training from score 382.028191


LGBMRegressor()

In [43]:
train_predictions_lgb_dafault = lgb_default_model.predict(X_train_scaled)
test_predictions_lgb_dafault = lgb_default_model.predict(X_test_scaled)
val_predictions_lgb_dafault = lgb_default_model.predict(X_val_scaled)

In [44]:
train_rmse_lgb_dafault = mean_squared_error(y_train, train_predictions_lgb_dafault, squared=False)
test_rmse_lgb_dafault = mean_squared_error(y_test, test_predictions_lgb_dafault, squared=False)
val_rmse_lgb_dafault = mean_squared_error(y_val, val_predictions_lgb_dafault, squared=False)

In [45]:
print(f"Training RMSE: {train_rmse_lgb_dafault}")
print(f"Test RMSE: {test_rmse_lgb_dafault}")
print(f"Val RMSE: {val_rmse_lgb_dafault}")

Training RMSE: 126.71615858634355
Test RMSE: 126.6701864455369
Val RMSE: 126.99565346390084


In [46]:
joblib.dump(lgb_default_model, '../../models/ANIKA/lgb_default_model.joblib')

['../../models/ANIKA/lgb_default_model.joblib']

In [47]:
#exp 4 LightGBM with best parameters

In [48]:
lgb_tuned_model = LGBMRegressor()
param_grid_lgbm = {
    'n_estimators': [150],  
    'max_depth': [3, 4, 5],          
    'learning_rate': [0.01, 0.1, 0.2]  
}

In [49]:
grid_search_LGB_tuned = GridSearchCV(lgb_tuned_model, param_grid_lgbm, cv=5, scoring='neg_mean_squared_error')
grid_search_LGB_tuned.fit(X_train_scaled, y_train)

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.085724 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 620
[LightGBM] [Info] Number of data points in the train set: 6879984, number of used features: 12
[LightGBM] [Info] Start training from score 382.051040
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.099075 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 620
[LightGBM] [Info] Number of data points in the train set: 6879984, number of used features: 12
[LightGBM] [Info] Start training from score 382.009346
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.082254 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memo

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.087220 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 620
[LightGBM] [Info] Number of data points in the train set: 6879984, number of used features: 12
[LightGBM] [Info] Start training from score 382.051040
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.124877 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 620
[LightGBM] [Info] Number of data points in the

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.130649 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 620
[LightGBM] [Info] Number of data points in the train set: 6879984, number of used features: 12
[LightGBM] [Info] Start training from score 382.009346
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGB

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.091109 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 620
[LightGBM] [Info] Number of data points in the train set: 6879984, number of used features: 12
[LightGBM] [Info] Start training from score 382.051040
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.082900 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 620
[LightGBM] [Info] Number of data points in the train set: 6879984, number of used features: 12
[Light

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad sinc

GridSearchCV(cv=5, estimator=LGBMRegressor(),
             param_grid={'learning_rate': [0.01, 0.1, 0.2],
                         'max_depth': [3, 4, 5], 'n_estimators': [150]},
             scoring='neg_mean_squared_error')

In [50]:
best_params_LGB_tuned = grid_search_LGB_tuned.best_params_
print("Best Hyperparameters:", best_params_LGB_tuned)

Best Hyperparameters: {'learning_rate': 0.2, 'max_depth': 5, 'n_estimators': 150}


In [51]:
lgb_tuned_model = LGBMRegressor(**best_params_LGB_tuned)
lgb_tuned_model.fit(X_train_scaled, y_train)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.121316 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 620
[LightGBM] [Info] Number of data points in the train set: 8599980, number of used features: 12
[LightGBM] [Info] Start training from score 382.028191
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No furt

LGBMRegressor(learning_rate=0.2, max_depth=5, n_estimators=150)

In [52]:
train_predictions_lgbm_tuned = lgb_tuned_model.predict(X_train_scaled)
test_predictions_lgbm_tuned = lgb_tuned_model.predict(X_test_scaled)
val_predictions_lgbm_tuned = lgb_tuned_model.predict(X_val_scaled)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).


In [53]:
train_rmse_lgbm_tuned = mean_squared_error(y_train, train_predictions_lgbm_tuned, squared=False)
test_rmse_lgbm_tuned = mean_squared_error(y_test, test_predictions_lgbm_tuned, squared=False)
val_rmse_lgbm_tuned = mean_squared_error(y_val, val_predictions_lgbm_tuned, squared=False)

In [54]:
print(f"Training RMSE: {train_rmse_lgbm_tuned}")
print(f"Test RMSE: {test_rmse_lgbm_tuned}")
print(f"Val RMSE: {val_rmse_lgbm_tuned}")

Training RMSE: 121.46773009197672
Test RMSE: 121.46045650243089
Val RMSE: 121.72659048866294


In [55]:
joblib.dump(lgb_tuned_model, '../../models/ANIKA/lgb_tuned_model.pkl')

['../../models/ANIKA/lgb_tuned_model.pkl']

In [ ]:
#exp 5 XGB with regularization

In [56]:
xgb_model_reg = XGBRegressor()

In [57]:
param_grid_reg = {
    'alpha': [0.1, 0.01, 0.001],
    'lambda': [1.0, 0.1, 0.01]
}

In [58]:
grid_search_reg = GridSearchCV(estimator=xgb_model_reg, param_grid=param_grid_reg, scoring='neg_mean_squared_error', cv=5)
grid_search_reg.fit(X_train_scaled, y_train)

GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None, gpu_id=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, m...in=None,
                                    max_cat_threshold=None,
                                    max_cat_to_onehot=None, max_delta_step=None,
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None, n_estimators=100,
                                    n_jobs=None, num_parallel_tree=None,
                                    predictor=None, random_state=None, ...),
             param_grid={'alpha': [0.1, 0.01, 0.001],
                         'lambda': [1.0, 0.1, 0.01]},
             scoring='neg_mean_squared_error')

In [59]:
best_alpha = grid_search_reg.best_params_['alpha']
best_lambda = grid_search_reg.best_params_['lambda']

In [60]:
best_xgb_model_reg = XGBRegressor(alpha=best_alpha, reg_lambda=best_lambda)

In [61]:
best_xgb_model_reg.fit(X_train_scaled, y_train)

XGBRegressor(alpha=0.001, base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, ...)

In [62]:
train_predictions_xgb_reg = best_xgb_model_reg.predict(X_train_scaled)
test_predictions_xgb_reg = best_xgb_model_reg.predict(X_test_scaled)
val_predictions_xgb_reg = best_xgb_model_reg.predict(X_val_scaled)

In [63]:
train_rmse_reg = mean_squared_error(y_train, train_predictions_xgb_reg, squared=False)
test_rmse_reg = mean_squared_error(y_test, test_predictions_xgb_reg, squared=False)
val_rmse_reg = mean_squared_error(y_val, val_predictions_xgb_reg, squared=False)

In [64]:
print(f"Training RMSE: {train_rmse_reg}")
print(f"Test RMSE: {test_rmse_reg}")
print(f"Val RMSE: {val_rmse_reg}")

Training RMSE: 115.89431762695312
Test RMSE: 115.97388458251953
Val RMSE: 116.27447509765625


In [65]:
joblib.dump(best_xgb_model_reg, '../../models/ANIKA/best_xgb_model_reg.joblib')

['../../models/ANIKA/best_xgb_model_reg.joblib']

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

# Splitting into features and target
X = final_data.drop('totalFare', axis=1)
y = final_data['totalFare']

# Label encoding categorical variables
label_encoders = {}
for col in ['startingAirport', 'destinationAirport', 'cabin_type', 'departure_time_category']:
    label_encoders[col] = LabelEncoder()
    X[col] = label_encoders[col].fit_transform(X[col])

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=True, random_state=23)

# Scale the data using StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
import joblib

joblib.dump(label_encoders, '../../models/ANIKA/label_encoder.joblib')
joblib.dump(scaler, '../../models/ANIKA/standard_scaler.joblib')

In [ ]:
X_train.shape, X_test.shape

In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error

# Calculate the mean of totalFare
mean_total_fare = y_train.mean()

# Create a list with the mean value to match the length of the test set
baseline_predictions = [mean_total_fare] * len(X_test)

# Calculate the Mean Absolute Error of the baseline model
rmse_baseline = np.sqrt(mean_squared_error(y_test, baseline_predictions))
print(f"Root Mean Squared Error of the baseline model: {rmse_baseline}")

In [ ]:
#path to save models